<a href="https://colab.research.google.com/github/joreeves/MaterialX-Library-Scraper/blob/master/mtlx_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#@title Settings
import os

gdrive = True #@param {type:"boolean"}
path = "/content/gdrive/MyDrive/mtlx/" #@param {type:"string"} 
unzip_files = True #@param {type:"boolean"}

if gdrive:
  from google.colab import drive
  drive.mount('/content/gdrive')

if not os.path.isdir(path):
  os.makedirs(path)

url = "https://matlib.gpuopen.com/main/materials/all"


Mounted at /content/gdrive


# Setup chromium-driver

solution from:
https://github.com/googlecolab/colabtools/issues/3347#issuecomment-1397277515

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.dyp3BXc6MC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.HWeXK0xnuC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.by1H51ZicK/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

# Utils

In [2]:
import time
import random
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def pause(max_time=20):
    time.sleep(random.randint(1, max_time))

def n_current_downloads(path_to_downloads):
    """Return the number of files currently downloading"""
    return len([f for f in os.listdir(path_to_downloads) if f.endswith('.crdownload')])

def download_wait(path_to_downloads, timeout=20, max_downloads=3):
    seconds = 0
    dl_wait = True
    while dl_wait:
        time.sleep(1)
        dl_wait = False

        n_dl = n_current_downloads(path_to_downloads)

        # If the number of downloads is greater than the max number of downloads
        # wait for the downloads to finish
        if (n_dl >= max_downloads):
            dl_wait = True
        elif (n_dl == 0):
            # If there are no downloads, wait for 1 second
            dl_wait = False
        elif (seconds >= timeout):
            # If the timeout is reached, wait for 1 second
            dl_wait = False
           
        seconds += 1

    return seconds

# Scrape

In [5]:
op = webdriver.ChromeOptions()
op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')
op.add_argument('--verbose')
op.add_argument("--disable-notifications")
op.add_experimental_option("prefs", {
  "download.default_directory": path,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True})
op.add_argument('--disable-gpu')
op.add_argument('--disable-software-rasterizer')

# Start driver

driver = webdriver.Chrome("/usr/bin/chromedriver", options=op)

driver.get(url)

wait = WebDriverWait(driver, 10)

wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "vue-recycle-scroller__item-view")))

pause()

<ipython-input-5-2cf4f7c7ebd3>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=op)


In [6]:
"""
This website splits elements into a grid based on window size
"""

# Get each row
rows = driver.find_elements(By.CLASS_NAME, "vue-recycle-scroller__item-view")

# Get the y position of each row
ypos = []
for r in rows:
    style = r.get_attribute("style")

    # find the integer in the style attribute
    # this is the y position of the row
    y = int(style.split('transform: translateY(')[-1].split("px);")[0])
    ypos.append(abs(y))

# find the index of the smallest y position in the list
# this is the first row
first_row = ypos.index(min(ypos))

In [7]:
# Get the first element in the first row
first_ele = rows[first_row].find_element(By.XPATH, "./div/div[1]")

# Click on the first element
first_ele.click()

# Wait for the download window to appear
wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="app"]/div[3]/div')))

pause()

In [ ]:
#@title Download the materials

# Get the download window
download_window = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div')

n_files  = int(driver.find_element(By.CLASS_NAME, 'v-chip__content').text)
pbar = tqdm(total=n_files)

scraping = True

while scraping:
    wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//*/table/tbody')))

    # Get the download table
    download_table = download_window.find_element(By.XPATH, '//*/table/tbody')

    # Get all of the rows in the table
    # Each row is a different resolution
    table_ele = download_table.find_elements(By.XPATH, "./tr[*]")

    # Get the last element in the table
    # This is the highest resolution
    last_ele = table_ele[-1].find_element(By.XPATH, "./td[3]/i")
    last_ele.click()

    pbar.update(1)

    download_wait(path)

    pause(20)

    # Click on the next button
    try:
        download_window.find_element(By.CLASS_NAME, 'mdi-chevron-double-right').click()
    except:
        scraping = False

In [ ]:
#@title Unzip the files

if unzip_files:
  zips = [f for f in os.listdir(path) if f.endswith('.zip')]

  for zip in zips:
    !unzip {path + zip}
    !rm {path + zip}